In [2]:
import numpy as np
import nibabel as nb
import pandas as pd
import scipy.io
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
Run1Seconds = np.array([i for i in range(819, 900+1)])
Run2Seconds = np.array([i for i in range(1737, 1818+1)])
Run3Seconds = np.array([i for i in range(2652, 2733+1)])
Run4Seconds = np.array([i for i in range(3553, 3634+1)])

# all frames

In [4]:
left_movie_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_left_Meanfile.mat')['Left_data']
right_movie_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']

In [5]:
left_movie_data.shape

(32492, 3655)

In [6]:
RunSeconds = np.array([Run1Seconds, Run2Seconds, Run3Seconds, Run4Seconds])

In [7]:
RunSeconds.shape

(4, 82)

In [9]:
delay = 4
RunSeconds_shifted = (RunSeconds + delay)%3655

In [10]:
RunSeconds_shifted[0].shape

(82,)

In [11]:
mean_runs_right_vertices = np.zeros((4, right_movie_data.shape[0]))
for i in range(4):
    mean_runs_right_vertices[i] = np.mean(right_movie_data[:, RunSeconds_shifted[i]], axis=1)

In [12]:
mean_runs_left_vertices = np.zeros((4, left_movie_data.shape[0]))
for i in range(4):
    mean_runs_left_vertices[i] = np.mean(left_movie_data[:, RunSeconds_shifted[i]], axis=1)

In [13]:
run2_minus_run1_right = mean_runs_right_vertices[1] - mean_runs_right_vertices[0]
run2_minus_run1_left = mean_runs_left_vertices[1] - mean_runs_left_vertices[0]

mean_run234_minus_run1_right = np.mean(mean_runs_right_vertices[1:4], axis=0) - mean_runs_right_vertices[0]
mean_run234_minus_run1_left = np.mean(mean_runs_left_vertices[1:4], axis=0) - mean_runs_left_vertices[0]

In [110]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/classic_place_ROIs/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)

temp_l.darrays[0].data = run2_minus_run1_left.astype(np.float32)
temp_l.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/run2_minus_run1_all_frames.L.func.gii')

temp_l.darrays[0].data = mean_run234_minus_run1_left.astype(np.float32)
temp_l.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/mean_run234_minus_run1_all_frames.L.func.gii')

In [111]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/classic_place_ROIs/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)

temp_r.darrays[0].data = run2_minus_run1_right.astype(np.float32)
temp_r.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/run2_minus_run1_all_frames.R.func.gii')

temp_r.darrays[0].data = mean_run234_minus_run1_right.astype(np.float32)
temp_r.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/mean_run234_minus_run1_all_frames.R.func.gii')

# place frames

In [41]:
RunSeconds = np.array([Run1Seconds, Run2Seconds, Run3Seconds, Run4Seconds])

In [42]:
file_path = 'E:/FPP_files_and_codes/FPP_files/relabeled_seconds/corrected_GPT_labeling/final_labels.csv'

RelabelDataframe = pd.read_csv(file_path)

In [43]:
RelabelDictionary = {}
for index, row in RelabelDataframe.iterrows():
    if index != 0:
        RelabelDictionary[row['prv_label']] = row['new_label']
RelabelDictionary[''] = float('nan')

In [44]:
labels_list_1 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie1_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_1.append(labels)
labels_list_2 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie2_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_2.append(labels)
labels_list_3 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie3_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_3.append(labels)
labels_list_4 = []
with open("E:/FPP_files_and_codes/FPP_files/HCP_movie_labels/Movie_labels/Movie4_labels.txt",'r') as file:
    for line in file:
        labels = line.strip().split(',')
        labels_list_4.append(labels)
labels_list = labels_list_1 + labels_list_2 + labels_list_3  + labels_list_4
print(len(labels_list))

3655


In [45]:
labels_list[793]

['playing_card', 'hand', 'flip']

In [46]:
RelabeldFrames = [set() for i in range(3655)]
for i in range(3655):
    for j in labels_list[i]:
        RelabeldFrames[i].add(str(RelabelDictionary[j]))

In [47]:
RelabeldFrames[846]

{'Body', 'Face', 'Object'}

In [50]:
RunsWithoutFB = np.empty(4, dtype=object)
for i in range(4):
    RunsWithoutFB[i] = []
for i in range(4):
    for j in RunSeconds[i]:
        if ('Place' in RelabeldFrames[j]) and (('Face' not in RelabeldFrames[j]) and ('Body' not in RelabeldFrames[j]) and ('Animal' not in RelabeldFrames[j])):
            RunsWithoutFB[i].append(j)

In [52]:
len(RunsWithoutFB[0])

23

In [53]:
PreferedSeconds = np.array([np.array(RunsWithoutFB[i]) for i in range(4)])

In [54]:
PreferedSeconds.shape

(4, 23)

In [55]:
delay = 4
ShiftedPreferedSeconds = (PreferedSeconds + delay)%3655

In [56]:
ShiftedPreferedSeconds.shape

(4, 23)

In [57]:
ShiftedPreferedSeconds

array([[ 823,  824,  825,  826,  827,  828,  829,  830,  831,  832,  833,
         834,  835,  836,  837,  838,  839,  875,  876,  881,  882,  883,
         896],
       [1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 1750, 1751,
        1752, 1753, 1754, 1755, 1756, 1757, 1793, 1794, 1799, 1800, 1801,
        1814],
       [2656, 2657, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2666,
        2667, 2668, 2669, 2670, 2671, 2672, 2708, 2709, 2714, 2715, 2716,
        2729],
       [3557, 3558, 3559, 3560, 3561, 3562, 3563, 3564, 3565, 3566, 3567,
        3568, 3569, 3570, 3571, 3572, 3573, 3609, 3610, 3615, 3616, 3617,
        3630]], dtype=int32)

In [58]:
mean_runs_right_vertices_place = np.zeros((4, right_movie_data.shape[0]))
for i in range(4):
    mean_runs_right_vertices_place[i] = np.mean(right_movie_data[:, ShiftedPreferedSeconds[i]], axis=1)

In [59]:
mean_runs_left_vertices_place = np.zeros((4, left_movie_data.shape[0]))
for i in range(4):
    mean_runs_left_vertices_place[i] = np.mean(left_movie_data[:, ShiftedPreferedSeconds[i]], axis=1)

In [60]:
run2_minus_run1_right_place = mean_runs_right_vertices_place[1] - mean_runs_right_vertices_place[0]
run2_minus_run1_left_place = mean_runs_left_vertices_place[1] - mean_runs_left_vertices_place[0]

mean_run234_minus_run1_right_place = np.mean(mean_runs_right_vertices_place[1:4], axis=0) - mean_runs_right_vertices_place[0]
mean_run234_minus_run1_left_place = np.mean(mean_runs_left_vertices_place[1:4], axis=0) - mean_runs_left_vertices_place[0]

In [61]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/classic_place_ROIs/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)

temp_l.darrays[0].data = run2_minus_run1_left_place.astype(np.float32)
temp_l.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/run2_minus_run1_place_frames.L.func.gii')

temp_l.darrays[0].data = mean_run234_minus_run1_left_place.astype(np.float32)
temp_l.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/mean_run234_minus_run1_place_frames.L.func.gii')

In [62]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/classic_place_ROIs/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)

temp_r.darrays[0].data = run2_minus_run1_right_place.astype(np.float32)
temp_r.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/run2_minus_run1_place_frames.R.func.gii')

temp_r.darrays[0].data = mean_run234_minus_run1_right_place.astype(np.float32)
temp_r.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/mean_run234_minus_run1_place_frames.R.func.gii')

# face and body frames

In [73]:
RunsWithoutP = np.empty(4, dtype=object)
for i in range(4):
    RunsWithoutP[i] = []
for i in range(4):
    for j in RunSeconds[i]:
        if (('Place' not in RelabeldFrames[j]) and (('Face' in RelabeldFrames[j]) or ('Body' in RelabeldFrames[j]) or ('Animal' in RelabeldFrames[j]))):
            RunsWithoutP[i].append(j)

In [74]:
PreferedSeconds = np.array([np.array(RunsWithoutP[i]) for i in range(4)])

In [75]:
len(RunsWithoutP[0])

20

In [76]:
PreferedSeconds.shape

(4, 20)

In [77]:
len(RunsWithoutP[0])

20

In [78]:
delay = 4
ShiftedPreferedSeconds = (PreferedSeconds + delay)%3655

In [79]:
ShiftedPreferedSeconds

array([[ 849,  850,  851,  852,  855,  856,  857,  858,  859,  860,  861,
         862,  863,  864,  865,  866,  867,  868,  900,  901],
       [1767, 1768, 1769, 1770, 1773, 1774, 1775, 1776, 1777, 1778, 1779,
        1780, 1781, 1782, 1783, 1784, 1785, 1786, 1818, 1819],
       [2682, 2683, 2684, 2685, 2688, 2689, 2690, 2691, 2692, 2693, 2694,
        2695, 2696, 2697, 2698, 2699, 2700, 2701, 2733, 2734],
       [3583, 3584, 3585, 3586, 3589, 3590, 3591, 3592, 3593, 3594, 3595,
        3596, 3597, 3598, 3599, 3600, 3601, 3602, 3634, 3635]],
      dtype=int32)

In [80]:
mean_runs_right_vertices_without_place = np.zeros((4, right_movie_data.shape[0]))
for i in range(4):
    mean_runs_right_vertices_without_place[i] = np.mean(right_movie_data[:, ShiftedPreferedSeconds[i]], axis=1)

In [81]:
mean_runs_left_vertices_without_place = np.zeros((4, left_movie_data.shape[0]))
for i in range(4):
    mean_runs_left_vertices_without_place[i] = np.mean(left_movie_data[:, ShiftedPreferedSeconds[i]], axis=1)

In [82]:
run2_minus_run1_right_without_place = mean_runs_right_vertices_without_place[1] - mean_runs_right_vertices_without_place[0]
run2_minus_run1_left_without_place = mean_runs_left_vertices_without_place[1] - mean_runs_left_vertices_without_place[0]

mean_run234_minus_run1_right_without_place = np.mean(mean_runs_right_vertices_without_place[1:4], axis=0) - mean_runs_right_vertices_without_place[0]
mean_run234_minus_run1_left_without_place = np.mean(mean_runs_left_vertices_without_place[1:4], axis=0) - mean_runs_left_vertices_without_place[0]

In [83]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/classic_place_ROIs/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)

temp_l.darrays[0].data = run2_minus_run1_left_without_place.astype(np.float32)
temp_l.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/run2_minus_run1_without_place_frames.L.func.gii')

temp_l.darrays[0].data = mean_run234_minus_run1_left_without_place.astype(np.float32)
temp_l.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/mean_run234_minus_run1_without_place_frames.L.func.gii')

In [84]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/classic_place_ROIs/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)

temp_r.darrays[0].data = run2_minus_run1_right_without_place.astype(np.float32)
temp_r.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/run2_minus_run1_without_place_frames.R.func.gii')

temp_r.darrays[0].data = mean_run234_minus_run1_right_without_place.astype(np.float32)
temp_r.to_filename('E:/FPP_files_and_codes/FPP_codes/Figure5_analysis/episodic_memory_analysis/mean_run234_minus_run1_without_place_frames.R.func.gii')

In [34]:
count = 0
for i in range(4):
    for j in RunSeconds[i]:
        if (('Place' not in RelabeldFrames[j]) and ('Face' not in RelabeldFrames[j]) and ('Body' not in RelabeldFrames[j]) and ('Animal' not in RelabeldFrames[j])):
            print(j)
            count += 1

849
850
884
885
886
891
1767
1768
1802
1803
1804
1809
2682
2683
2717
2718
2719
2724
3583
3584
3618
3619
3620
3625


In [33]:
count

24

In [40]:
RelabeldFrames[849]

{'Object', 'Unclear'}

In [39]:
labels_list[849]

['web_page', 'text', 'choose', 'cursor']